In [806]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [807]:
# display multiple outputs in one row
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [808]:
connection = Connection()
db = connection.recipe_db
input_data = db.recipes_without_reviews

data = json.loads(json_util.dumps(input_data.find()))

In [809]:
ing = pd.DataFrame(json_normalize(data, record_path='ingredients',
                             meta='id', record_prefix='ingredients_', errors='ignore'))


nutritions = pd.DataFrame(json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count']))


In [811]:
#  ------  erstellung und data cleansing - Ingredients

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
# setze multiinde auf 'id' und 'ingredients_id'
ingredients = ing.set_index(['id', 'ingredients_id'])

# filtere alle Zutaten samt ihrer rezepte id, die weniger gleich 5 mal vorkommen
ingredients_eqles_5_ing = ingredients.groupby(
    'ingredients_id').filter(lambda x: len(x) <= 5)

# droppe alle rezepte, die eine Zutate besitzen, die weniger gleich 5 Mal vorkommen
ingredients_filt = ingredients.drop(ingredients_eqles_5_ing.index.get_level_values('id').values, level=0)

# drop alls rows with ingredients_id == 0
ingredients_eqal_zero = ingredients_filt[ingredients_filt.index.get_level_values('ingredients_id') == 0]
ingredients_filt = ingredients_filt.drop(ingredients_eqal_zero.index.get_level_values('id').values, level=0)


In [813]:
#  ------ Erstellung und cleansing des Nutrition Dataframes

# erstelle neue liste auf basis der bereits gefilterten rezepte aus ingredients_filt
id_overlap_mask = nutritions['id'].isin(ingredients_filt.index.get_level_values('id').values)

# erstelle datenframe auf basis der overlapliste
nutritions_filt = nutritions.loc[id_overlap_mask]

nutrition_db = nutritions_filt.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)

# remove multiindex 'amount'
nutrition_db.columns = nutrition_db.columns.droplevel(0)

# entferne alle NA
nutrition_db = nutrition_db.dropna()

# gleiche nochmals die ids der beiden dataframe nutrition und ingredients ab, da der nutrition dataframe noch NA Werte hatt
id_overlap_mask = ingredients_filt.index.get_level_values('id').isin(nutrition_db.index)
ingredients_db = ingredients_filt[id_overlap_mask]

# abgleich ob anzahl der indizes von nutrition und zutaten dataframe gleich sind


In [815]:


ingredients_db.reset_index(inplace=True)

recipe_db = pd.get_dummies(ingredients_db['ingredients_id']).groupby(
    ingredients_db['id']).apply(max)



In [817]:
new_ingredients = ingredients_db.copy()
#new_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index()
gramms_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index().copy()

Q1 = gramms_ingredients.quantile(0.25)
Q3 = gramms_ingredients.quantile(0.75)
IQR = Q3 - Q1

#Filter out all recipes which are outlier by their weight (gramms)
df = gramms_ingredients[~((gramms_ingredients < (Q1 - 1.5 * IQR)) |(gramms_ingredients > (Q3 + 1.5 * IQR))).any(axis=1)].copy()

#filter out recipes by weight which are not in the range 500 - 2373.59 gramms
df_start_at_fivehundret = df[df['ingredients_grams'].between(500, 2373.58225, inclusive=False)].copy()

df_start_at_fivehundret.set_index('id', inplace=True)
id_overlap_mask = nutritions['id'].isin(df_start_at_fivehundret.index.get_level_values('id').values)


# erstelle datenframe auf basis der overlapliste
nutritions_filt_gramm = nutritions.loc[id_overlap_mask]

nutrition_db2 = nutritions_filt_gramm.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

#create new nutrition db based on the above filtering
nutrition_db2.set_index('id', inplace=True)
nutrition_db2.columns = nutrition_db2.columns.droplevel(0)



In [819]:
cluster2_ids = [  7198,   8494,   8539,   8545,   8590,   8614,   8639,   8667,
         8680,   8726,   8819,   8886,   8890,   8919,   8934,   8950,
         8991,   9009,   9021,   9023,   9025,   9051,   9231,   9252,
         9337,  11690,  11904,  12026,  12710,  12744,  12783,  12837,
        13199,  13861,  13974,  14001,  14044,  14495,  14502,  14521,
        14526,  14611,  14624,  14679,  14680,  14689,  14697,  14714,
        14740,  14758,  14759,  14840,  14951,  15005,  15224,  15646,
        15679,  15867,  16119,  16174,  16254,  16329,  16362,  16415,
        16449,  16683,  16862,  17092,  17325,  17511,  17619,  17749,
        17792,  17803,  17855,  17938,  17949,  18039,  18101,  18286,
        18340,  18837,  18874,  19509,  19929,  20040,  20095,  20415,
        20474,  20593,  20768,  20779,  21151,  21186,  21201,  21230,
        21561,  21700,  21815,  22219,  22297,  22299,  22331,  22340,
        22351,  23567,  23632,  23723,  23814,  24010,  24034,  24743,
        25150,  25151,  25260,  25312,  25469,  25475,  25491,  25859,
        25867,  26591,  26618,  26627,  26651,  27819,  27953,  28015,
        28035,  28363,  28366,  29455,  29460,  31026,  31799,  31813,
        31963,  35200,  35554,  35892,  36124,  36856,  37689,  37965,
        38046,  39374,  40450,  40574,  40604,  41636,  41661,  42609,
        42968,  43057,  44149,  45381,  45580,  45688,  45833,  46402,
        46530,  46819,  47015,  47455,  48074,  48546,  49014,  50007,
        50233,  50235,  50579,  50658,  50726,  50870,  50939,  51039,
        51436,  52299,  52446,  53404,  54611,  54702,  55826,  56140,
        57429,  57716,  57761,  57966,  58097,  58165,  58941,  60155,
        60333,  62825,  67001,  68129,  68382,  69030,  69620,  69660,
        69744,  69754,  70463,  70585,  70679,  71145,  71402,  72007,
        72804,  74698,  74708,  75811,  75861,  76705,  77408,  77515,
        77524,  77942,  78131,  78144,  78189,  78649,  79543,  79901,
        80742,  80827,  80883,  83117,  83792,  84014,  84142,  84675,
        84886,  85919,  86733,  86860,  87217,  87535,  88404,  88495,
        89852,  96698, 100386, 100606, 102603, 104704, 107586, 109544,
       116006, 125366, 125394, 128601, 130444, 132351, 132934, 134471,
       137414, 138041, 138118, 140920, 141716, 143064, 143082, 145843,
       148831, 148910, 150258, 150308, 158638, 159188, 160234, 160241,
       160957, 162392, 162438, 162807, 169616, 170566, 171368, 172704,
       175773, 176359, 182107, 185519, 186685, 186910, 188286, 188688,
       188957, 190015, 190094, 191156, 191199, 192070, 201010, 201849,
       201851, 201964, 202469, 212396, 212915, 213227, 213482, 213749,
       214217, 214227, 214535, 214544, 215026, 217799, 217962, 218070,
       218072, 218075, 218078, 218604, 218941, 219193, 219454, 220124,
       220134, 220260, 220347, 221069, 221950, 222342, 222712, 222934,
       223047, 223100, 223101, 223156, 223588, 228266, 228285, 228619,
       228630, 228640, 228823, 228872, 228917, 228944, 229690, 230250,
       230818, 230905, 230981, 230995, 231004, 231229, 231230, 231381,
       231493, 231768, 232335, 232409, 232591, 233243, 233446, 233975,
       235476, 236162, 236284, 236609, 237807, 237842, 237927, 238073,
       240400, 240708, 240954, 241165, 241601, 244507, 257312]

cluster2 = nutrition_db2[nutrition_db2.index.isin(cluster2_ids)]

In [1630]:
# random
subset_random_top10 = nutrition_db2.sample(n=10).copy()
subset_random_top10.index.to_numpy()


array([ 17373,  17730,  24798, 220493,  11838,  23920,  21140,  45833,
       142056,  48734])

In [1136]:
subset_normal = cluster2.sample(n=40).copy()
subset_normal_id = subset_normal.index.to_numpy()

In [1138]:
# remove recipes which are missing in nutrition df from the ingredients tecipes dataframe
mask = recipe_db.index.isin(nutrition_db2.index.values)
recipe_db = recipe_db.loc[mask]


In [1299]:
user_recipes_normal = recipe_db[recipe_db.index.isin(subset_normal_id)]

In [1300]:
#top10 most common ingredients
#16421    2125 salt
#4342     1506 garlic
#4397     1412 onion
#16406    1185 ground black pepper
#16157    1016 butter
#6307      944 olive oil
#6494      786 skinless bone less chicken breast
#2496      749 water
#16238     574 grated Parmesan cheese
#16317     538 eggs


drop_id_list = [16421, 4342, 4397, 16406, 16157, 6307, 6494, 2496, 16238, 16317]



In [1302]:
#filter out ingredients which are too common and the recipes in the user profile from the recipes database
new_recipe_db = recipe_db.drop(axis=1, labels=drop_id_list)


new_recipe_db_wo_userrecipes_normal = new_recipe_db.drop(axis=0, labels=subset_normal_id)


In [1303]:
#remove basic ingredients like garlic, olive oil, skinless bone less chicken breast, butter, milk, salt and pepper, onion
new_user_recipes_normal = user_recipes_normal.drop(axis=1, labels=drop_id_list)

In [1524]:
sample_normal = new_user_recipes_normal.sample(n=40).copy()

In [1525]:
## Jaccard - normal

result_array = cdist(sample_normal, new_recipe_db_wo_userrecipes_normal,'jaccard')
result_w_filter_normal = pd.DataFrame(result_array, columns=new_recipe_db_wo_userrecipes_normal.index.values, index=sample_normal.index.values)
# result_2

result_w_filter_10_normal = pd.DataFrame(result_w_filter_normal.sum(), columns=['jaccard_distance_sum'])
result_w_filter_10_normal = result_w_filter_10_normal.sort_values(by='jaccard_distance_sum')
result_w_filter_10_normal['jaccard_distance_sum'] = result_w_filter_10_normal['jaccard_distance_sum'].div(10)
#result_w_filter_10_normal[0:10]


In [1526]:
result_w_filter_normal_short = result_w_filter_normal.copy()

In [1527]:
# top 10 normal short
normal_short = pd.DataFrame(np.sort(result_w_filter_normal_short.values, axis=0), index=result_w_filter_normal_short.index, columns=result_w_filter_normal_short.columns)

new_columns = normal_short.columns[normal_short.loc[normal_short.first_valid_index()].argsort()]
result_normal_short = normal_short[new_columns]
result_normal_short = result_normal_short.reset_index()
result_normal_short = result_normal_short.loc[0].to_frame()
#result_normal_short[0:11].index.to_numpy()


In [1528]:
#create one dimensional ingredients vector from user recipes profile
user_one_vector_normal = sample_normal.copy()
user_one_vector_normal.loc['sum',:]= user_one_vector_normal.sum(axis=0)
user_one_vector_normal = user_one_vector_normal.drop(axis=0, labels=sample_normal.index)
user_one_vector_normal[user_one_vector_normal > 0] = 1

In [1529]:
#user_one_vector_normal.apply(pd.value_counts).count(axis=1)

In [1530]:
## Jaccard - Vector normal top 10

result_array = cdist(user_one_vector_normal, new_recipe_db_wo_userrecipes_normal,'jaccard')
result_w_filter_vector_normal = pd.DataFrame(result_array, columns=new_recipe_db_wo_userrecipes_normal.index.values, index=user_one_vector_normal.index.values)


result_w_filter_vector_10_normal = pd.DataFrame(result_w_filter_vector_normal.sum(), columns=['jaccard_distance_sum'])
result_w_filter_vector_10_normal = result_w_filter_vector_10_normal.sort_values(by='jaccard_distance_sum')

#result_w_filter_vector_10_normal[0:10]


In [1532]:
####### results

In [1533]:
# userrecipes normal
sample_normal.index.to_numpy()

array([188957, 229690, 218604,   9025, 233975,  12783,  17749, 230250,
       107586,  57429, 231381, 240954,  22351,  55826,  13199, 236609,
        16415, 132351, 222712, 141716,  78189,  22299,  14526, 220347,
        57716,  17619, 109544,  70463,  75811,  48546, 170566, 143064,
       202469,  16254, 223156,  24743,  19929, 191199,  58097, 236284])

In [1534]:
###### jaccard normal top 10 naiv
result_w_filter_10_normal[0:10].index.to_numpy()

array([138041,  67001, 100606, 241165,  96698,   8819,  17092,   8919,
       201849,  13974])

In [1535]:
###### jaccard kurz normal top 10
result_normal_short[1:11].index.to_numpy()


array([233243, 138041, 50579, 96698, 23814, 74698, 100606, 79901, 16174,
       8614], dtype=object)

In [1536]:
###### jaccard vector normal top 10
result_w_filter_vector_10_normal[0:10].index.to_numpy()

array([ 52299, 221069,  51039,  22331, 223100, 237807,  14758, 231396,
        77942, 228266])

In [1537]:
##### random top 10
subset_random_top10.index.to_numpy()

array([138041, 228266, 214544,  57429,  89852, 228823,  46402,   9009,
        68129,  42609])